### Example notebook to inspect a set of targetids with Prospect

E. Armengaud - A. Raichoor

In the context of minisv2, tile-based data, March 2020

The code to find spectra associated to a given targetid is "homemade"... (and will change for sure)
 .. But it's ok to use it to inspect specific targets in minisv2 !

In [69]:
# Load modules :
# - use DESI master kernel
# - get prospect from desihub : https://github.com/desihub/prospect/tree/minisv
#     use the minisv branch (as of early March)
#     setup pythonpath, eg :
import sys, os, glob
sys.path.append("/global/homes/a/armengau/prospect/py")
from prospect import plotframes,utils_specviewer,myspecselect
import desispec.io
from astropy.table import Table, vstack
import numpy as np
from astropy.io import fits

In [70]:
# Two homemade functions, defined within the notebook at this stage.

# 1) Make "mini-db" of targetids. It basically reads zbest, so decently it's fast.
def make_targetdict(tiledir, petals=[str(i) for i in range(10)], tiles_in_seven=True) :
    mydict = {}
    tiles = os.listdir(tiledir)
    if tiles_in_seven : tiles = [x for x in tiles if x[0]=='7']
    for tile in tiles:
        nights = os.listdir(os.path.join(tiledir,tile))
        for night in nights :
            mydict[tile+"-"+night] = {}
            # exposures
            fns = np.sort(glob.glob(os.path.join(tiledir,tile,night,'cframe-b'+petals[0]+'-????????.fits')))
            mydict[tile+"-"+night]['exps'] = np.array([fn.replace('.','-').split('-')[-2] for fn in fns])
            # targetid, fibres
            targetid,fiber,petal_list = [],[],[]
            for petal in petals:
                pp = glob.glob(os.path.join(tiledir,tile,night,'zbest-'+petal+'-'+tile+'-????????.fits'))
                if len(pp)>0 :
                    fn        = pp[0]
                    data      = fits.open(fn)['fibermap'].data
                    targetid += data['targetid'].tolist()
                    fiber    += data['fiber'].tolist()
                    petal_list    += [petal for i in range(len(data['targetid']))]
            mydict[tile+"-"+night]['targetid'] = np.array(targetid)
            mydict[tile+"-"+night]['fiber']    = np.array(fiber)
            mydict[tile+"-"+night]['petal']    = np.array(petal_list)            
    return mydict

# 2) Make spectra/zcat from a targetlist.
def make_spectra_zcat_from_targets(targets, tiledir, obs_db) :
    spectra = None
    ztables = []
    for target in targets :
        for tile_night in obs_db.keys() :
            w, = np.where( obs_db[tile_night]['targetid'] == target )
            if len(w) > 1 : print("?? Pbl ??")
            if len(w) == 0 : continue
            the_path = tile_night.replace("-","/")
            the_petal = obs_db[tile_night]['petal'][w[0]]
            the_spec = desispec.io.read_spectra(os.path.join(tiledir,the_path,"coadd-"+the_petal+"-"+tile_night+".fits"))
            the_spec = myspecselect.myspecselect(the_spec, targets=[target], remove_scores=True)
            the_spec.fibermap['EXPID'] = the_spec.fibermap['FIBER']*0 # trick for Spectra.update()
            if the_spec.num_spectra()!=1 : print("!! Pbl .. ??")
            ztables.append(Table.read(os.path.join(tiledir,the_path,"zbest-"+the_petal+"-"+tile_night+".fits"),'ZBEST'))
        if spectra is None : spectra = the_spec
        else : spectra.update(the_spec)
    zcat = vstack(ztables)
    return (spectra,zcat)

In [71]:
# Make "mini-db" 
tiledir   = '/global/cfs/cdirs/desi/spectro/redux/daily/tiles/'
petals = ['0','3','6','7','9']
obs_db = make_targetdict(tiledir, petals=petals)

In [72]:
## Enter list of targets HERE. 
# In this example we select some random targets from the minidb
targets = [ obs_db['70002-20200304']['targetid'][34], obs_db['70005-20200303']['targetid'][2] ]

# Make spectra+zcat
spectra, zcat= make_spectra_zcat_from_targets(targets, tiledir, obs_db)

In [ ]:
# VI command in notebook
plotframes.plotspectra(spectra, zcatalog=zcat, title='Targetid selection', notebook=True, mask_type='CMX_TARGET')